# Generate reference solution

In [1]:
import numpy as np
import torch
from torch import nn
from typing import Tuple
import import_ipynb
import json

In [2]:
def read_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

In [3]:
torch.set_default_dtype(torch.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_file = "Initial_NN.torch"

In [4]:
from Classes_Beam import Beam, Prob_Solv_modes, In_Cond

importing Jupyter notebook from Classes_Beam.ipynb


In [5]:
par = read_json('par.json')

Lx = par['x']
t = par['t']
n = par['n']

num_hidden = par['n_layers']
dim_hidden = par['n_neurons']

In [6]:
my_beam = Beam(Lx, 68e9, 2700, 8e-3, 40e-3, n)

In [7]:
prob = Prob_Solv_modes(my_beam)

gamma_max = 100 # gamma_max must be increased, because spatial eigenfrequencies increase, since the beam is very short

prob.pass_g_max(gamma_max)
eig_gam = prob.find_eig()

my_beam.gamma = np.array(eig_gam)
my_beam.update_freq()

# Just one parameter independent for gamma (order of the system reduced)
F = prob.find_all_F(my_beam)
prob.update_gamma(my_beam)
phi = prob.return_modemat(F)
my_beam.update_phi(phi)

In [8]:
my_In_Cond = In_Cond(my_beam)

w_0 = my_beam.phi[:,0]
w_dot_0 = np.zeros(len(w_0))

my_In_Cond.pass_init_cond(w_0, w_dot_0)
A, B = my_In_Cond.compute_coeff()

t = np.linspace(0, 1, n)
my_beam.calculate_solution(A, B, t)
w = my_beam.w

In [9]:
def adimensionalize_sol(w: np.ndarray, w_ast: float):
    return w/w_ast

w_ad = adimensionalize_sol(w, Lx).T

correct to transpose it? based on how reshape works
- columns of w are referred to different time points

# Setup NN for initial conditions

## Define training dataset

In [10]:
x = np.linspace(0, 1, n)

X, T = np.meshgrid(x, t)
X = X.reshape(-1, 1)
T = T.reshape(-1, 1)

x = torch.tensor(X).to(device).float()
t = torch.tensor(T).to(device).float()

[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [1.]
 [1.]]


## Prepare data for pytorch

In [11]:
y = torch.tensor(w_ad).float()
y = y.reshape(-1, 1).to(device)

In [12]:
from Classes_NN import *

importing Jupyter notebook from Classes_NN.ipynb


In [13]:
nn = NN(num_hidden, dim_hidden, dim_input = 2, dim_output = 1)

In [14]:
from tqdm import tqdm
from typing import Callable
import pytz

epochs = 8000
lr = 0.01

loss_fn = Loss(
    x,
    t,
    y
)

In [15]:
from datetime import datetime

pinn_trained, loss_values = train_model(
    nn, loss_fn=loss_fn, learning_rate=lr, max_epochs=epochs)

Loss: 2.6209: : 8001it [01:03, 125.48it/s]                        


In [16]:
import os

def obtain_date():
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%B %d - %H:%M")    
    return formatted_datetime

folder = 'in_model'
model_name = f'{obtain_date()}.pth'
model_path = os.path.join(folder, model_name)
os.makedirs(folder, exist_ok=True)

torch.save(nn.state_dict(), model_path)

# To be checked
- ~correct association of meshpoints with solution?~